In [1]:
import numpy as np
from Molecule import Molecule
np.set_printoptions(precision=7, linewidth=120, suppress=True)


In [2]:
mol = Molecule('input/h2o/STO-3G/geom.dat')
nuclear_replusion = 'input/h2o/STO-3G/enuc.dat'
one_electron = 'input/h2o/STO-3G/s.dat'
kinetic_energy =  'input/h2o/STO-3G/t.dat'
nuclear_attraction =  'input/h2o/STO-3G/v.dat'
two_electron = 'input/h2o/STO-3G/eri.dat'

# mol = Molecule('input/h2o/STO-3G/geom.dat')
# nuclear_replusion = 'input/h2o/DZP/enuc.dat'
# one_electron = 'input/h2o/DZP/s.dat'
# kinetic_energy =  'input/h2o/DZP/t.dat'
# nuclear_attraction =  'input/h2o/DZP/v.dat'
# two_electron = 'input/h2o/DZP/eri.dat'


 

# Nuclear Repulsion Energy

In [3]:
with open(nuclear_replusion, 'r') as fp:
    nr = float(fp.readline().strip())

print(f'Nuclear repulsion energy = {nr}')

Nuclear repulsion energy = 8.00236706181045


In [4]:
print('Overlap Integral = ')
overlap_int =  mol.read_intregals(one_electron)
print(overlap_int)

print('Kinetic Energy Integral = ')
kinetic_int =  mol.read_intregals(kinetic_energy)
print(kinetic_int)

print('Nuclear Attraction  Integral = ')
nuclear_int =  mol.read_intregals(nuclear_attraction)
print(nuclear_int)

print('Core Hamiltonian = ')
H_core = kinetic_int + nuclear_int
print(H_core)

Overlap Integral = 
[[ 1.         0.2367039 -0.        -0.        -0.         0.0384056  0.0384056]
 [ 0.2367039  1.         0.        -0.         0.         0.3861388  0.3861388]
 [-0.         0.         1.        -0.         0.         0.2684382 -0.2684382]
 [-0.        -0.        -0.         1.        -0.         0.2097269  0.2097269]
 [-0.         0.         0.        -0.         1.        -0.        -0.       ]
 [ 0.0384056  0.3861388  0.2684382  0.2097269 -0.         1.         0.1817599]
 [ 0.0384056  0.3861388 -0.2684382  0.2097269 -0.         0.1817599  1.       ]]
Kinetic Energy Integral = 
[[29.0031999 -0.1680109  0.         0.        -0.        -0.0084164 -0.0084164]
 [-0.1680109  0.808128   0.        -0.         0.         0.0705174  0.0705174]
 [ 0.         0.         2.5287312 -0.         0.         0.1470909 -0.1470909]
 [ 0.        -0.        -0.         2.5287312 -0.         0.11492    0.11492  ]
 [-0.         0.         0.        -0.         2.5287312 -0.        -0. 

# Build the Orthogonal Matrix

In [5]:
# Diagonalize the overlap matrix:
eigvalue, eigvector = np.linalg.eigh(overlap_int)

#Build the symmetric orthogonalization matrix using:

S_ortho = eigvector @ np.diag(eigvalue**(-1/2)) @ eigvector.T

print('The S^-1/2 matrix is = ')
print(S_ortho)

# Check for orthogonality
print(S_ortho.dot(overlap_int).dot(S_ortho))



The S^-1/2 matrix is = 
[[ 1.0236346 -0.1368547  0.        -0.0074873  0.         0.0190279  0.0190279]
 [-0.1368547  1.1578632 -0.         0.0721601 -0.        -0.2223326 -0.2223326]
 [ 0.        -0.         1.0733148 -0.        -0.        -0.1757583  0.1757583]
 [-0.0074873  0.0721601 -0.         1.038305  -0.        -0.1184626 -0.1184626]
 [ 0.        -0.        -0.        -0.         1.        -0.         0.       ]
 [ 0.0190279 -0.2223326 -0.1757583 -0.1184626 -0.         1.1297234 -0.0625975]
 [ 0.0190279 -0.2223326  0.1757583 -0.1184626  0.        -0.0625975  1.1297234]]
[[ 1.  0.  0. -0.  0.  0.  0.]
 [ 0.  1.  0. -0. -0.  0.  0.]
 [ 0.  0.  1. -0. -0. -0. -0.]
 [-0. -0. -0.  1. -0. -0. -0.]
 [ 0. -0. -0. -0.  1. -0.  0.]
 [ 0. -0. -0. -0. -0.  1. -0.]
 [ 0. -0. -0. -0.  0. -0.  1.]]


# Build the Initial Guess Density 

In [6]:
# F_p = (S_ortho.T)@ H_core @ S_ortho
F = S_ortho.dot(H_core).dot(S_ortho)
print('Initial Fock Matrix = ')
print(F)
e, C_p = np.linalg.eigh(F)


C = S_ortho.dot(C_p)

print('Initial C matrix: ')
print(C)

# Density Matrix

nocc = int(mol.occ)

C_occ = C[:, :nocc]
# D_int = C_0[:, :5] @ C_0[:, :5].T 
D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

print('Initial Density Matrix : ')
print(D)






Initial Fock Matrix = 
[[-32.2545866  -2.7914909  -0.          0.008611    0.         -0.1812967  -0.1812967]
 [ -2.7914909  -8.2368891   0.         -0.2282926  -0.         -0.3857987  -0.3857987]
 [ -0.          0.         -7.542889    0.          0.         -0.1132121   0.1132121]
 [  0.008611   -0.2282926   0.         -7.4570295  -0.         -0.1102196  -0.1102196]
 [  0.         -0.          0.         -0.         -7.3471449   0.         -0.       ]
 [ -0.1812967  -0.3857987  -0.1132121  -0.1102196   0.         -4.0329547  -0.0446466]
 [ -0.1812967  -0.3857987   0.1132121  -0.1102196  -0.         -0.0446466  -4.0329547]]
Initial C matrix: 
[[-1.0015436 -0.2336245 -0.         0.0856842 -0.        -0.0482226  0.       ]
 [ 0.0071893  1.0579388  0.        -0.3601105  0.         0.4631213 -0.       ]
 [-0.        -0.         1.0610702  0.         0.        -0.        -0.2965071]
 [ 0.0002671  0.4272843 -0.         0.9399425 -0.         0.2129401 -0.       ]
 [ 0.         0.        -0. 

In [7]:
# Build the Fock Matrix 
SCF_E = 0.0
E_old = 0.0 
E_conv = 1.0e-12

for scf_iter in range(1, 1000):
    J = np.einsum("uvkl, kl -> uv", mol.read_2e_intregals(two_electron), D)
    K = np.einsum("uvkl, kl -> uv", mol.read_2e_intregals(two_electron), D)
    Fock = H_core + 2*J - K

    SCF_E = np.einsum('pq,pq->', ( H_core + Fock), D, optimize=True) + nr
    print(f'SCF Iteration {scf_iter}   Energy {SCF_E}   dE {SCF_E - E_old}')

    if (abs(SCF_E - E_old) <   E_conv) :
        break
    E_old = SCF_E

    # Compute new orbital guess
    F_p = S_ortho.dot(Fock).dot(S_ortho)
    e, C_p = np.linalg.eigh(F_p)
    C = S_ortho.dot(C_p)

    C_occ = C[:, :nocc]

    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))
    

SCF Iteration 1   Energy -90.57072065762084   dE -90.57072065762084
SCF Iteration 2   Energy -90.72898303865524   dE -0.1582623810344046
SCF Iteration 3   Energy -90.72997471582393   dE -0.000991677168684646
SCF Iteration 4   Energy -90.72998558817893   dE -1.0872355005631107e-05
SCF Iteration 5   Energy -90.7299857037481   dE -1.1556916490462754e-07
SCF Iteration 6   Energy -90.72998570498072   dE -1.2326211162871914e-09
SCF Iteration 7   Energy -90.72998570499384   dE -1.311661890213145e-11
SCF Iteration 8   Energy -90.729985704994   dE -1.5631940186722204e-13

SCF converged.
Final RHF Energy: -90.72998570 [Eh]


# Compute the Inital SCF Energy

In [8]:
E_0 = np.einsum('pq,pq->', (H_core + F), D, optimize=True) + nr

print(E_0)

-119.02850863091038
